In [2]:

import shap
import xgboost
import numpy as np
import pandas as pd
import statsmodels.api as sm
import category_encoders as ce

from category_encoders import CountEncoder
from category_encoders import WOEEncoder

from geopy.geocoders import Nominatim

from sklearn import preprocessing
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.decomposition import PCA

pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [3]:
df_raw = pd.read_csv("train.csv")

In [4]:
df_raw.columns

Index(['property_id', 'property_name', 'property_summary', 'property_space',
       'property_desc', 'property_neighborhood', 'property_notes',
       'property_transit', 'property_access', 'property_interaction',
       'property_rules', 'property_zipcode', 'property_lat', 'property_lon',
       'property_type', 'property_room_type', 'property_max_guests',
       'property_bathrooms', 'property_bedrooms', 'property_beds',
       'property_bed_type', 'property_amenities', 'property_sqfeet',
       'property_scraped_at', 'property_last_updated', 'host_id', 'host_since',
       'host_location', 'host_about', 'host_response_time',
       'host_response_rate', 'host_nr_listings', 'host_nr_listings_total',
       'host_verified', 'booking_price_covers', 'booking_min_nights',
       'booking_max_nights', 'booking_availability_30',
       'booking_availability_60', 'booking_availability_90',
       'booking_availability_365', 'booking_cancel_policy', 'reviews_num',
       'reviews_first', 're

Featurization: 
host location: 
    =>extract city and country: first and last of string separated by comma
    => Bolean if same city as property
    =>Boolean if same country as property
revieuws: recency
zip code, long and lat
    => find city and country to check if same as host: boolean columns https://pypi.org/project/pgeocode/
    https://stackoverflow.com/questions/69409255/how-to-get-city-state-and-country-from-a-list-of-latitude-and-longitude-coordi
    OR find long lat of host and estimate the distance to the property
    => could even find distance to city centre using these methods
    => use use counts in data or weights of evidence to encode these https://contrib.scikit-learn.org/category_encoders/count.html#
    => maybe find a way of combining the three: the are obviously correlated!! 
    => start by using the zip code, then, the location within a city becomes important=> maybe 
    => or use one or the other and see which one is better
property_type=many categories: words may mean the same thing in different areas
    => categorize them or use weights of evidence etc
property_room_type: categorize
property beds, etc: max_guests or beds/ bedrooms or bathrooms or square feet=> difference between dormitory and luxurious apartment
scraped_at&last updated: find time since
host since: find time since
host location: City=> Bolean if same location as property


'booking_availability_30'... obviously highly correlated: could also use a ration to get it between 0 and 1 => beta's for all durations are comparable
may want to use some form of word mining to get the most out of the descriptions. But I'll skip for now

booking_cancel_policy: turn categories into numbers, order is important

In [9]:
print(df_raw['booking_cancel_policy'].unique())

['flexible' 'moderate' 'strict' 'super_strict_30']


This creates a column for recency. Can be used for each date since column

In [ ]:
# Define a lambda function to calculate the recency of each date value
recency_func = lambda x: (pd.datetime.today() - x).days

# Convert the 'date' column to a datetime object
df_raw['reviews_last'] = pd.to_datetime(df_raw['reviews_last'])
# Apply the lambda function to the 'date' column to create a new 'recency' column
df_raw['recency'] = df_raw['reviews_last'].apply(recency_func)


Creating recency variables
#also add scraped_at....

In [ ]:
recency_variables=["reviews_last" ,"reviews_first" ,"host_since"]
recency_columns=["recency_"+ i for i in recency_variables]
print(recency_variables)
print(recency_columns)

for var, new_var in zip(recency_variables, recency_columns):
    df_raw[var] = pd.to_datetime(df_raw[var])
    recency_func = lambda x: (pd.datetime.today() - x).days
    df_raw[new_var] = df_raw[var].apply(recency_func)
    print(df_raw[new_var].head())

This encodes using counts

In [70]:
# create a list of variables on which this can be used and list with encoded variable names
count_list=["property_type","property_room_type", "property_zipcode"]
count_encode_list=["count_encoded"+ i for i in count_list]
for var, encoded_var in zip(count_list, count_encode_list):
    # Create an instance of the CountEncoder class

    encoder = CountEncoder()
    # Fit the encoder to the column
    encoder.fit(df_raw[var])
    # Transform the column using the encoder
    encoded_column = encoder.transform(df_raw[var])
    # Add the encoded column to the original DataFrame
    df_raw[encoded_var] = encoded_column


print(count_encode_list)
print(df_raw['count_encodedproperty_type'].head())

Cell below not important, just for review

In [76]:
# Fit the encoder to the column
encoder.fit(df_raw['property_zipcode'])
# Add the encoded column to the original DataFrame
df_raw['encoded_property_zipcode'] = encoder.transform(df_raw['property_zipcode'])

Encoding using weights of evidence=> but shouldn't really be used for continuous variables and code doesn't work

In [ ]:
# create a list of variables on which this can be used and list with encoded variable names
count_list=["property_type","property_room_type", "property_zipcode"]
WOE_encode_list=["WOE_"+ i for i in count_list]
for var, encoded_var in zip(count_list, count_encode_list):
    # Create an instance of the CountEncoder class

    encoder = WOEEncoder()
    # Fit the encoder to the column
    encoder.fit(df_raw[var],df_raw['target'])
    # Add the encoded column to the original DataFrame
    df_raw[encoded_var] = encoder.transform(df_raw[var])


print(WOE_encode_list)
print(df_raw['WOEE_property_type'].head())


Finding ratio of max guest or beds per bathroom, bedroom or square meters

Below does not work yet!!
find distance between 2 coordinate points

ip code, long and lat
    => find city and country to check if same as host: boolean columns https://pypi.org/project/pgeocode/
    https://stackoverflow.com/questions/69409255/how-to-get-city-state-and-country-from-a-list-of-latitude-and-longitude-coordi
    OR find long lat of host and estimate the distance to the property
    => could even find distance to city centre using these methods
    => use use counts in data or weights of evidence to encode these https://contrib.scikit-learn.org/category_encoders/count.html#
    => maybe find a way of combining the three: the are obviously correlated!! 
    => start by using the zip code, then, the location within a city becomes important=> maybe 
    => or use one or the other and see which one is better

In [61]:
#distance between two coordinate points

import geopy.distance

coords_1 = (52.2296756, 21.0122287)
coords_2 = (52.406374, 16.9251681)

print geopy.distance.geodesic(coords_1, coords_2).km


>>> from geopy.distance import geodesic
>>> newport_ri = (41.49008, -71.312796)
>>> cleveland_oh = (41.499498, -81.695391)
>>> print(geodesic(newport_ri, cleveland_oh).miles)
538.390445368

SyntaxError: invalid syntax (1066247191.py, line 8)

In [ ]:
from geopy.geocoders import Nominatim
address='Nagpur'
geolocator = Nominatim(user_agent="Sander")
location = geolocator.geocode(address)
#print(location.address)
print((location.latitude, location.longitude))
location=location.latitude
print(location)

In [98]:
print(df_raw['host_location'])
df_raw[host_city]=
for item in df_raw['host_location']:
    df_raw['host_city']=

0                             Brussels, Brussels, Belgium
1                              Antwerp, Flanders, Belgium
2                              Antwerp, Flanders, Belgium
3                                                 Belgium
4                                                 Belgium
                              ...                        
6490                          Brussels, Brussels, Belgium
6491    Aix-en-Provence, Provence-Alpes-Côte d'Azur, F...
6492                        Keerbergen, Flanders, Belgium
6493                   Antwerpen, Flemish Region, Belgium
6494                   Antwerpen, Flemish Region, Belgium
Name: host_location, Length: 6495, dtype: object
